In [1]:
print("hellow hord")

hellow hord


In [3]:
%pip install --upgrade langchain langchain-google-genai google-generativeai


     ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
     ------------------------ --------------- 41.0/68.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 68.0/68.0 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 1.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-stat


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\crish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

API_IA = os.getenv("GOOGLE_API_KEY")

Conexão com o Gemini

In [19]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0,
    api_key = API_IA
)

In [21]:
resp_text = llm.invoke("o que é agentes em IA ?")
print(resp_text.content)

Em Inteligência Artificial (IA), um **agente** é uma entidade (seja um software ou um hardware) que **percebe** seu **ambiente** através de **sensores** e **age** sobre esse ambiente através de **efetores**, com o objetivo de alcançar **metas** ou maximizar uma **medida de desempenho**.

Pense nele como um "ser" inteligente que interage com o mundo ao seu redor.

Vamos quebrar os componentes principais:

1.  **Percepção (Perception):** É a capacidade do agente de receber informações do seu ambiente.
    *   **Sensores:** São os meios pelos quais o agente percebe.
        *   *Exemplos para um agente de software:* Entradas de teclado, dados de um banco de dados, pixels de uma imagem, sinais de áudio, pacotes de rede.
        *   *Exemplos para um agente de hardware (robô):* Câmeras, microfones, sensores de toque, sensores de distância.

2.  **Ambiente (Environment):** É o "mundo" no qual o agente opera e interage.
    *   *Exemplos:* O mundo físico (para um robô), um jogo de computador,

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."

    """
    decisao: AUTO_RESOLVER
    urgencia: BAIXA
    campo_faltante: []
    """
)